In [13]:
from perceiver.data import *
import utils
import numpy as np

In [66]:
def maskMetric(PD, GT):
    PD = PD.reshape(1, -1)
    GT = GT.reshape(1, -1)
    TP, TN, FP, FN = 0, 0, 0, 0
    for i in range(GT.shape[0]):
        P = GT[i].sum()
        TP += (PD[i][GT[i] == 1] == 1).sum()
        TN += (PD[i][GT[i] == 0] == 0).sum()
        FP += (PD[i][GT[i] == 0] == 1).sum()
        FN += (PD[i][GT[i] == 1] == 0).sum()
    return np.array([float(TP), float(TN), float(FP), float(FN)])

In [15]:
dm = CosmicDataModule(batch_size=100, root="../datasets")
dm.setup(stage=None)
x_test, y_test = utils.dm_to_numpy(dm.test_dataloader())

../datasets/cosmic
../datasets/cosmic/npy_train/f435w/9490/a3/2_208.npy


In [25]:
dl = dm.test_dataloader()


In [34]:
imgs, masks, ignores = [], [], []
for batch in dl:
    img, mask, ignore = batch
    imgs.append(img.numpy())
    masks.append(mask.numpy())
    ignores.append(ignore.numpy())


In [38]:
imgs = np.concatenate(imgs)
masks = np.concatenate(masks)
ignores = np.concatenate(ignores)

In [39]:
print(imgs.shape)

(4830, 128, 128)


# Load the model

In [71]:
import xgboost as xgb

def eval_cosmic_seed(seed):
    model = xgb.XGBClassifier()
    model.load_model(f"../xgboost_model_cosmic_{seed}.json")
    logits = model.predict([x.reshape(-1) for x in imgs]).reshape(*imgs.shape)
    tp, tn, fp, fn = maskMetric(
        logits.reshape(-1, 1, 128, 128) > 0.5, 
        masks.reshape(-1, 1, 128, 128))

    fpr = fn / (fn + tp)
    return fpr

In [73]:
res = [eval_cosmic_seed(i) for i in range(3)]
res

[0.46172886435650445, 0.46226738011205, 0.46388732701721225]

In [79]:
res = np.array(res) * 100
mean = np.mean(res)
std = np.std(res)

print(mean, std)

46.26278571619222 0.09173142108099631
